In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

# 비재무 정보와 주가 정보 merge

In [22]:
non_finance = pd.read_excel('./data/비재무 it 토탈.xlsx')

In [23]:
stock_price = pd.read_excel('./data/비재무 주가 토탈.xlsx')

In [24]:
total = pd.merge(non_finance, stock_price, how = 'left', on = ['회사명','거래소코드','회계년도'])

In [25]:
nc = total[total['회사명'] == '(주)엔씨소프트']
sdi = total[total['회사명'] == '삼성SDI(주)']
ssem = total[total['회사명'] == '삼성전기(주)']
hynix = total[total['회사명'] == '에스케이하이닉스(주)']

# 자료 정렬

In [26]:
nc = nc.dropna(axis = 0)
sdi = sdi.dropna(axis = 0)
ssem = ssem.dropna(axis = 0)
hynix = hynix.dropna(axis = 0)

In [27]:
nc.sort_values('회계년도', ascending= True, inplace = True)
nc.reset_index(drop = True, inplace = True)

In [28]:
sdi.sort_values('회계년도', ascending= True, inplace = True)
sdi.reset_index(drop = True, inplace = True)

In [29]:
ssem.sort_values('회계년도', ascending= True, inplace = True)
ssem.reset_index(drop = True, inplace = True)

In [30]:
hynix.sort_values('회계년도', ascending= True, inplace = True)
hynix.reset_index(drop = True, inplace = True)

# 상관관계(절대값) 만들기

In [31]:
nc_corr = nc.corr(method = 'pearson')
nc_corr['종가(원)'] = abs(nc_corr['종가(원)'])
df_nc = pd.DataFrame(nc_corr['종가(원)']).reset_index().sort_values('종가(원)', ascending = False)
df_nc.columns = ['지표','종가(원)']

sdi_corr = sdi.corr(method = 'pearson')
sdi_corr['종가(원)'] = abs(sdi_corr['종가(원)'])
df_sdi = pd.DataFrame(sdi_corr['종가(원)']).reset_index().sort_values('종가(원)', ascending = False)
df_sdi.columns = ['지표','종가(원)']

ssem_corr = ssem.corr(method = 'pearson')
ssem_corr['종가(원)'] = abs(ssem_corr['종가(원)'])
df_ssem = pd.DataFrame(ssem_corr['종가(원)']).reset_index().sort_values('종가(원)', ascending = False)
df_ssem.columns = ['지표','종가(원)']

hynix_corr = hynix.corr(method = 'pearson')
hynix_corr['종가(원)'] = abs(hynix_corr['종가(원)'])
df_hynix = pd.DataFrame(hynix_corr['종가(원)']).reset_index().sort_values('종가(원)', ascending = False)
df_hynix.columns = ['지표','종가(원)']

In [32]:
pd.concat([hynix_corr['종가(원)'],sdi_corr['종가(원)'],ssem_corr['종가(원)'],nc_corr['종가(원)']],axis = 1)

,종가(원),종가(원),종가(원),종가(원)
거래소코드,NaN,NaN,NaN,NaN
보통주식수(3개월)(주),0.672984,0.370457,0.008597,0.039502
보통주식비율(3개월),0.327265,0.147585,0.007435,0.039873
우선주식수(3개월)(주),NaN,0.337889,0.288067,NaN
우선주식비율(3개월),NaN,0.337066,0.118365,NaN
관리사무직_기말인원_남(명),NaN,NaN,NaN,NaN
관리사무직_기말인원_여(명),NaN,NaN,NaN,NaN
관리사무직_기말인원(명),NaN,NaN,NaN,NaN
관리사무직_년간평균급여(천원),NaN,NaN,NaN,NaN
관리사무직_1인평균년간급여(천원),NaN,NaN,NaN,NaN


In [37]:
list_corr = []
order = 15 # 각 기업의 상관관계 높은 15개에서 공통된 값 뽑기 ( order 갯수 변경해보자 )

for i in range(order):
    list_corr.append(df_nc.index[:order][i])
    list_corr.append(df_sdi.index[:order][i])
    list_corr.append(df_ssem.index[:order][i])
    list_corr.append(df_hynix.index[:order][i])

variable = pd.Series(list_corr).value_counts()
variable = pd.DataFrame(variable).reset_index()
variable.columns = ['var_index', 'number']

In [38]:
var = []
for i in range(len(variable)):
    var.append(df_nc.reset_index()['지표'][variable.index[i]])
var = pd.DataFrame(var)
var.columns = ['var']

In [39]:
pd.concat([df_nc.reset_index(),df_sdi.reset_index(),df_ssem.reset_index(),df_hynix.reset_index()],axis =1)

,index,지표,종가(원),index,지표,종가(원),index,지표,종가(원),index,지표,종가(원)
0,40,종가(원),1.000000,40,종가(원),1.000000,40,종가(원),1.000000,40,종가(원),1.000000
1,39,산술평균(원),0.993929,39,산술평균(원),0.970195,39,산술평균(원),0.985698,37,최고가(원),0.991909
2,37,최고가(원),0.987233,38,최저가(원),0.967548,37,최고가(원),0.983848,39,산술평균(원),0.989041
3,38,최저가(원),0.986987,37,최고가(원),0.966322,38,최저가(원),0.979403,38,최저가(원),0.975185
4,17,정규+무기계약_기말인원(명),0.897190,22,기간제_기말인원(명),0.584030,42,거래대금(원),0.800499,42,거래대금(원),0.870778
5,32,합계_기말인원(명),0.897043,15,정규+무기계약_기말인원(남)(명),0.489534,41,거래량(주),0.594732,15,정규+무기계약_기말인원(남)(명),0.721815
6,15,정규+무기계약_기말인원(남)(명),0.801769,17,정규+무기계약_기말인원(명),0.485425,21,기간제_기말인원(여)(명),0.585554,30,합계_기말인원_남(명),0.721634
7,30,합계_기말인원_남(명),0.793160,42,거래대금(원),0.458283,22,기간제_기말인원(명),0.578830,17,정규+무기계약_기말인원(명),0.696614
8,16,정규+무기계약_기말인원(여)(명),0.677745,30,합계_기말인원_남(명),0.451053,30,합계_기말인원_남(명),0.518774,32,합계_기말인원(명),0.691834
9,31,합계_기말인원_여(명),0.661241,32,합계_기말인원(명),0.449580,15,정규+무기계약_기말인원(남)(명),0.494286,1,보통주식수(3개월)(주),0.672984


In [40]:
corr = pd.concat([variable,var], axis = 1)
corr = corr[['var_index','var','number']]
corr.head(20) # 52번까지 재무비율

,var_index,var,number
0,42,종가(원),4
1,32,산술평균(원),4
2,15,최고가(원),4
3,17,최저가(원),4
4,22,정규+무기계약_기말인원(명),4
5,30,합계_기말인원(명),4
6,37,정규+무기계약_기말인원(남)(명),4
7,40,합계_기말인원_남(명),4
8,38,정규+무기계약_기말인원(여)(명),4
9,39,합계_기말인원_여(명),4


# ----------------------------------------------------------------------
## <font color='red'> 변수 선정까지만 하고 딥러닝을 통한 모델은 안 만듬
# ----------------------------------------------------------------------

# 주가와 높은 상관관계를 가진 변수간 상관관계 보기

In [ ]:
# nc_corr[corr['var'][:20]]

# 상관관계 시각화

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib import font_manager, rc
# import seaborn as sns
# import matplotlib
# font_location = "c:/Windows/fonts/malgun.ttf"
# font_name = font_manager.FontProperties(fname=font_location).get_name()
# matplotlib.rc('font', family=font_name)

In [ ]:
# plt.figure(figsize=(20,10))

# sns.heatmap(data = nc_corr[corr['var'][:20]], annot = True, cmap = 'bwr')
# # plt.savefig('tmp.png')

# 여기서는 산업별 변수가 아닌 nc 만을 고려함!!!!
# 산업별 변수 써서 해도 됨!!!

In [ ]:
df_nc

In [ ]:
nc_corr['매도가능금융자산 평가손익(IFRS연결)(천원)'].sort_values(ascending = False)

In [ ]:
# nc_corr.to_excel("tmp.xlsx")

# 딥러닝 모델 구축 - 여기서는 산업별 변수가 아닌 nc 만을 고려함!!!!

In [ ]:
# 재무피쳐값 입력
# corr.loc[1:13,'var'] => 이거는 산업별 변수임

In [ ]:
# 재무 피쳐값 입력 및 종가 입력
nc_deep = nc[['매도가능금융자산평가손익(IFRS연결)(천원)','이익잉여금(결손금)(*)(IFRS연결)(천원)','기초자본금(IFRS연결)(천원)',
             '매도가능금융자산 평가손익(IFRS연결)(천원)', '기말자본금(*)(IFRS연결)(천원)']]

nc_deep['종가'] = nc['종가']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

# minmax_scale(nc_deep)
nc_deep = pd.DataFrame(scale(nc_deep))

In [ ]:
import tensorflow as tf
from tqdm import tqdm_notebook
tf.set_random_seed(777)  # for reproducibility

x_data = nc_deep.loc[:,nc_deep.columns[:-1]]
y_data = nc_deep.loc[:,[nc_deep.columns[-1]]] # 괄호를 한번 더 씌어야 한다는 거 주의!!!!!!


# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, len(x_data.columns)])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([len(x_data.columns), 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [ ]:
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in tqdm_notebook(range(600001)):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})

#     과정보기
#     if step % 1000 == 0 or step < 10 :
#         print("\nStep : {} \nCost : {} \nPrediction :\n{}".format(step, cost_val, hy_val))


In [ ]:
nc_deep.iloc[:,-1]

In [ ]:
real_price = (150500 + (447500-150500)*(nc_deep.iloc[:,-1] -nc_deep.iloc[:,-1][0])/(nc_deep.iloc[:,-1][10]-nc_deep.iloc[:,-1][0])) # 실제값
real_price

In [ ]:
predicted_price = pd.Series((150500 + (447500-150500)*(hy_val - nc_deep.iloc[:,-1][0])/(nc_deep.iloc[:,-1][10]-nc_deep.iloc[:,-1][0])).ravel()) # 예상값
predicted_price

In [ ]:
(real_price - predicted_price) / real_price

모든 변수로 예측한 값

0     148217.979945  
1     182266.349690  
2     198259.848990  
3     184924.459174  
4     221566.862975  
5     221690.388927  
6     192670.366688  
7     202617.361196  
8     266704.364747  
9     389411.266116  
10    432578.433438  
11    376595.414338  